# Import the libraries

In [ ]:
import pandas as pd         #to work with cols
import numpy as np
import os                   #to interact with the operating system
import re                   #to work with text (regular expressions)

In [47]:
pd.set_option('display.max_colwidth', None) #quiero ver todo el texto de more info
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Reading the file

In [244]:
input_file = os.path.join('..', 'data', 'UK_Sanctions_List.xlsx')
df= pd.read_excel(input_file, header= 1)

In [ ]:
df.sort_values(by= 'Group ID', ascending= True)

# View the data

In [ ]:
print(df.columns.tolist())

['Name 6', 'Name 1', 'Name 2', 'Name 3', 'Name 4', 'Name 5', 'Title', 'Name Non-Latin Script', 'Non-Latin Script Type', 'Non-Latin Script Language', 'DOB', 'Town of Birth', 'Country of Birth', 'Nationality', 'Passport Number', 'Passport Details', 'National Identification Number', 'National Identification Details', 'Position', 'Address 1', 'Address 2', 'Address 3', 'Address 4', 'Address 5', 'Address 6', 'Post/Zip Code', 'Country', 'Other Information', 'Group Type', 'Alias Type', 'Alias Quality', 'Regime', 'Listed On', 'UK Sanctions List Date Designated', 'Last Updated', 'Group ID']


In [178]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18796 entries, 0 to 18795
Data columns (total 38 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Name 6                             18796 non-null  object        
 1   Name 1                             12584 non-null  object        
 2   Name 2                             7255 non-null   object        
 3   Name 3                             1098 non-null   object        
 4   Name 4                             254 non-null    object        
 5   Name 5                             35 non-null     object        
 6   Title                              1058 non-null   object        
 7   Name Non-Latin Script              3990 non-null   object        
 8   Non-Latin Script Type              1359 non-null   object        
 9   Non-Latin Script Language          1497 non-null   object        
 10  DOB                               

In [151]:
# Ver qué columnas tienen valores nulos y el porcentaje de valores nulos
nulls = df.isnull().sum() / len(df) * 100
nulls[nulls > 0].sort_values(ascending=False)

Name 5                             99.813790
Name 4                             98.648649
Address 4                          95.919345
Title                              94.371143
Name 3                             94.158332
Non-Latin Script Type              92.769738
Non-Latin Script Language          92.035539
National Identification Details    91.854650
Address 5                          91.588636
Address 3                          89.449883
National Identification Number     89.242392
Passport Details                   88.800809
Post/Zip Code                      87.055757
Passport Number                    85.236220
Address 2                          81.315173
Name Non-Latin Script              78.772079
Alias Quality                      74.420089
Address 1                          65.508619
Name 2                             61.401362
Position                           58.916791
Address 6                          54.942541
Town of Birth                      50.409662
Country of

In [188]:
df['Group ID'].nunique()

4686

# Transformations

In [ ]:
# Transform date columns to datetime type
df['DOB'] = pd.to_datetime(df['DOB'], format='%d/%m/%Y', errors='coerce')
df['Listed On'] = pd.to_datetime(df['Listed On'], format='%d/%m/%Y')
df['UK Sanctions List Date Designated'] = pd.to_datetime(df['UK Sanctions List Date Designated'], format='%d/%m/%Y')
df['Last Updated'] = pd.to_datetime(df['Last Updated'], format='%d/%m/%Y')

In [ ]:
# join the names and adresses
df['full_name'] = df[['Name 6', 'Name 1', 'Name 2', 'Name 3', 'Name 4', 'Name 5']].fillna('').agg(' '.join, axis=1).str.strip() #join the names
df['Full Address'] = df[['Address 1', 'Address 2', 'Address 3', 'Address 4', 'Address 5', 'Address 6']].fillna('').agg(' '.join, axis= 1).str.strip()
# unify the names 
df['full_name'] = df['full_name'].str.title() 
df['Full Address'] = df['Full Address'].replace('', np.nan)


In [182]:
def get_primary_name(group):    #name in 'Primary name'
    primary_row = group[group['Alias Type'] == 'Primary name']
    
    if not primary_row.empty:
        return primary_row['full_name'].iloc[0]
    else: # first full_name of the group
        return group['full_name'].iloc[0]
    
def get_aliases(group): #other names
    names = group['full_name'].dropna().unique().tolist()
    primary = get_primary_name(group)
    aliases = [name for name in names if name != primary]
    return aliases

def get_addresses(group): 
    addresses = group['Full Address'].dropna().unique().tolist()
    return addresses

def get_related_countries(group): #including birth country and other related countries
    countries = group['Country'].dropna().unique().tolist()
    return countries

#other info
def get_other_info(group):
    return group['Other Information'].iloc[0] if 'Other Information' in group else None

def get_nationality(group):
    return group['Nationality'].iloc[0]  if 'Nationality' in group else None

def get_regime(group):
    return group['Regime'].iloc[0]  if 'Regime' in group else None

#all the dates
def get_dates(group):
    dates = {
        'DOB': group['DOB'].dropna().iloc[0] if not group['DOB'].isna().all() else None,
        'listed_on': group['Listed On'].dropna().iloc[0] if not group['Listed On'].isna().all() else None,
        'uk_sanctions_list_date_designated': group['UK Sanctions List Date Designated'].dropna().iloc[0] if not group['UK Sanctions List Date Designated'].isna().all() else None,
        'last_updated': group['Last Updated'].dropna().iloc[0] if not group['Last Updated'].isna().all() else None
    }
    return dates


# agg by 'group_id', apply .iloc and call all the def
grouped_df = df.groupby('Group ID').apply(lambda group: pd.Series({
    'group_id': group.name,
    'group_type': group['Group Type'].dropna().iloc[0] if not group['Group Type'].isna().all() else pd.NA,
    'primary_name': get_primary_name(group),
    'aliases': get_aliases(group),
    'addresses': get_addresses(group),
    'regime': get_regime(group),
    'related_countries': get_related_countries(group),
    'nationality': get_nationality(group),
    'other_information': get_other_info(group),
    'dates': get_dates(group)
})).reset_index(drop=True)

# each date by column
dates_df = pd.json_normalize(grouped_df['dates'])
grouped_df = pd.concat([grouped_df.drop(columns='dates'), dates_df], axis=1)

# let's check :D
grouped_df.head(6)

C:\Users\ethel\AppData\Local\Temp\ipykernel_15640\4083169379.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_df = df.groupby('Group ID').apply(lambda group: pd.Series({


,group_id,group_type,primary_name,aliases,addresses,regime,related_countries,nationality,other_information,DOB,listed_on,uk_sanctions_list_date_designated,last_updated
0,6894,Individual,Abdurrahman Mohamad Iqbal,"[Abdurrahman Abu Jibril, Iqbal A Rahman Mohamad, Iqbal Abdul Rahman Mohamad, Iqbal Rahman Mohamad, Muqti Fihiruddin, Muqti Fikiruddin]",[Jalan Nakula Komplek Witana Harja III Blok C 106-107 Tangerang],ISIL (Da'esh) and Al-Qaida,[Indonesia],Indonesia,(UK Sanctions List Ref):AQD0235. (UN Ref):QDi.086. Review pursuant to Security Council resolution 1822 (2008) was concluded on 8 Jun. 2010. Review pursuant to Security Council resolution 2368 (2017) was concluded on 4 Dec. 2019. INTERPOL-UN Security Council Special Notice web link: https://www.interpol.int/en/How-we-work/Notices/View-UN-Notices-Individuals.,1958-08-17,2003-01-24,2003-01-28,2023-02-08
1,6895,Individual,Abdul Qader Abdul Hai Hazem,[Hazem Abdul Hai],[Puli Charkhi Area District Number 9 Kabul City Kabul Province],Afghanistan,[Afghanistan],Afghanistan,(UK Sanctions List Ref):AFG0110. (UN Ref):TAi.142. Review pursuant to Security Council resolution 1822 (2008) was concluded on 29 Jul. 2010. INTERPOL-UN Security Council Special Notice web link: https://www.interpol.int/en/How-we-work/Notices/View-UN-Notices-Individuals click here,NaT,2001-02-23,2001-01-25,2021-02-01
2,6897,Individual,Agha Abdul Manan,"[Al-Man Saiyid Abd, Manan Abdul, Saiyid Abdul Man'Am]",[],ISIL (Da'esh) and Al-Qaida,[],NaN,(UK Sanctions List Ref):AQD0104. (UN Ref):QDi.018. Pakistan. Review pursuant to Security Council resolution 1822 (2008) was concluded on 15 Jun. 2010.Review pursuant to Security Council resolution 2253 (2015) was concluded on 21 Feb. 2019.INTERPOL-UN Security Council Special Notice web link: https://www.interpol.int/en/notice/search/un/1423806,NaT,2001-10-12,2001-10-17,2020-12-31
3,6899,Individual,Shihata Tharwat Salah,"[Abdallah Tarwat Salah, Ali Tharwat Salah Shihata, Thirwat Salah Shihata, Thirwat Shahata]",[],ISIL (Da'esh) and Al-Qaida,[],Egypt,(UK Sanctions List Ref):AQD0327. (UN Ref):QDi.017. Review pursuant to Security Council resolution 1822 (2008) was concluded on 15 Jun. 2010. Review pursuant to Security Council resolution 2368 (2017) was concluded on 4 Dec. 2019. INTERPOL-UN Security Council Special Notice web link: www.interpol.int/en/How-we-work/Notices/View-UN-Notices-Individuals.,1960-06-29,2001-12-06,2001-10-06,2020-12-31
4,6901,Individual,Abdul Chaudhry Majeed,"[Abdul Majeed, Abdul Majeed Chaudhry, Abdul Majid]",[],ISIL (Da'esh) and Al-Qaida,[],Pakistan,(UK Sanctions List Ref):AQD0222. (UN Ref):QDi.054. Reportedly deceased. Review pursuant to Security Council resolution 1822 (2008) was concluded on 1 Jun. 2010. Review pursuant to Security Council resolution 2253 (2015) was concluded on 21 Feb. 2019. INTERPOL-UN Security Council Special Notice web link: https://www.interpol.int/en/notice/search/un/1422960,1939-04-15,2001-12-24,2001-12-24,2023-02-08
5,6904,Individual,Awal Shah Abdul Baqi Basir,[Baqi Abdul],[],Afghanistan,[],Afghanistan,(UK Sanctions List Ref):AFG0034. (UN Ref):TAi.038. Believed to be in Afghanistan/Pakistan border area. Taliban member responsible for Nangarhar Province as at 2008. Until 7 Sep. 2007 he was also listed under number TAi.048. Review pursuant to Security Council resolution 1822 (2008) was concluded on 1 Jun. 2010. INTERPOL-UN Security Council Special Notice web link: https://www.interpol.int/en/How-we-work/Notices/View-UN-Notices-Individuals click here,NaT,2001-04-02,2001-02-23,2021-02-01


# Analysis of grouped_df 

In [191]:
grouped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4686 entries, 0 to 4685
Data columns (total 13 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   group_id                           4686 non-null   int64         
 1   group_type                         4686 non-null   object        
 2   primary_name                       4686 non-null   object        
 3   aliases                            4686 non-null   object        
 4   addresses                          4686 non-null   object        
 5   regime                             4686 non-null   object        
 6   related_countries                  4686 non-null   object        
 7   nationality                        2594 non-null   object        
 8   other_information                  4686 non-null   object        
 9   DOB                                2584 non-null   datetime64[ns]
 10  listed_on                          4

In [193]:
# Ver qué columnas tienen valores nulos y el porcentaje de valores nulos
nulls = grouped_df.isnull().sum() / len(grouped_df) * 100
nulls.sort_values(ascending=False)

DOB                                  44.857021
nationality                          44.643619
primary_name                          0.000000
group_type                            0.000000
group_id                              0.000000
addresses                             0.000000
aliases                               0.000000
related_countries                     0.000000
regime                                0.000000
other_information                     0.000000
listed_on                             0.000000
uk_sanctions_list_date_designated     0.000000
last_updated                          0.000000
dtype: float64

In [205]:
grouped_df[(grouped_df['nationality'].isna()) & (grouped_df['group_type']== 'Individual')]

,group_id,group_type,primary_name,aliases,addresses,regime,related_countries,nationality,other_information,DOB,listed_on,uk_sanctions_list_date_designated,last_updated
2,6897,Individual,Agha Abdul Manan,"[Al-Man Saiyid Abd, Manan Abdul, Saiyid Abdul Man'Am]",[],ISIL (Da'esh) and Al-Qaida,[],NaN,(UK Sanctions List Ref):AQD0104. (UN Ref):QDi.018. Pakistan. Review pursuant to Security Council resolution 1822 (2008) was concluded on 15 Jun. 2010.Review pursuant to Security Council resolution 2253 (2015) was concluded on 21 Feb. 2019.INTERPOL-UN Security Council Special Notice web link: https://www.interpol.int/en/notice/search/un/1423806,NaT,2001-10-12,2001-10-17,2020-12-31
79,7099,Individual,Bin Marwan Bilal,[],[],ISIL (Da'esh) and Al-Qaida,[],NaN,(UK Sanctions List Ref):AQD0156. (UN Ref):QDi.009. Senior lieutenant of UBL. Review pursuant to Security Council resolution 1822 (2008) was concluded on 21 Jun. 2010. Review pursuant to Security Council resolution 2253 (2015) was concluded on 21 Feb. 2019. INTERPOL-UN Security Council Special Notice web link: https://www.interpol.int/en/notice/search/un/1423806,NaT,2001-02-23,2001-01-25,2020-12-31
203,7591,Individual,Al-Rawi Saif-Al-Din Fulayyih Hassan Taha,[Al-Rawi Ayad Futayyih],[],Iraq,[],NaN,(UK Sanctions List Ref):IRQ0075. (UN Ref):IQi.014.,NaT,2003-07-02,2003-06-27,2020-12-31
260,7886,Individual,Hamdan Usama,[],[],Counter-Terrorism (International),[],NaN,"(UK Sanctions List Ref):CTI0018. (UK Statement of Reasons):Usama Hamdan is a senior official of Hamas and a member of the group’s politburo. He is therefore a member of and associated with Hamas, and threatened retaliation and violence against Israel. (Gender):Male",NaT,2004-03-24,2020-12-31,2022-02-16
261,7887,Individual,Mishaal Khalid,[],[],Counter-Terrorism (International),[],NaN,"(UK Sanctions List Ref):CTI0019. (UK Statement of Reasons):Khalid Mishaal was the leader of Hamas from 2004 until 2014 and maintains a leadership role in the Shura Council of Hamas. He has directed the group’s violent attacks, including the targeting of civilians. (Gender):Male",NaT,2004-03-24,2020-12-31,2020-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4610,16773,Individual,Abramov Vasily Vitalyevich,[],[],Russia,[],NaN,"(UK Sanctions List Ref):RUS2397. Financial sanctions imposed in addition to an asset freeze: Trust services. Date trust services sanctions imposed: 24/02/2025. (UK Statement of Reasons):VASILY VITALYEVICH ABRAMOV is or has been involved in destabilising Ukraine or undermining or threatening the territorial integrity, sovereignty or independence of Ukraine by working as a director, trustee or other manager or equivalent position of KAZSTANEX LIMITED LIABILITY PARTNERSHIP (KAZSTANEX LLP). KAZSTANEX LLP is an involved person and is or has been involved in destabilizing Ukraine or undermining or threatening the territorial integrity, sovereignty or independence of Ukraine, through providing financial services or making available funds, economic resources, goods or technology that could contribute to destabilising Ukraine or undermining or threatening the territorial integrity, sovereignty or independence of Ukraine. \n (Email address):v.abramov@kazstanex.kz (Gender):Male",1982-05-23,2025-02-24,2025-02-24,2025-02-27
4677,16847,Individual,Parasca Natalia,[],[],Global Anti-Corruption,[],NaN,"(UK Sanctions List Ref):GAC0073. (UK Statement of Reasons):Natalia PARASCA (“PARASCA”) is an involved person under the Global Anti-Corruption Sanctions Regulations 2021 on the basis of the following grounds: (1) PARASCA is acting on behalf of or at the direction of Ilan Shor; (2) PARASCA is associated with Ilan Shor. Ilan Shor is an involved person under the Global Anti-Corruption Sanctions Regulations 2021 on the basis that he has been involved in serious corruption, namely being responsible for or engaging in serious corruption with respect to bribery of foreign public officials.\n (Gender):Female",NaT,2025-04-02,2025-04-02,2025-04-0

In [206]:
print("Valores únicos en la columna 'Country':")
print(df['Country'].unique())

Valores únicos en la columna 'Country':
['Pakistan' nan 'Kazakhstan' 'Syria' 'Myanmar' 'Belarus' 'Russia'
 'United Kingdom' 'North Korea' 'Singapore' 'United Arab Emirates' 'Iran'
 'Libya' 'United States' 'Lebanon' 'Tunisia' 'Greece'
 'Central African Republic' 'Germany' 'Iraq' 'Italy' 'Algeria' 'Turkey'
 'Sudan' 'Iraq (previous address)' 'Syrian Arab Republic' 'Afghanistan'
 'Somalia' 'Kenya' 'Philippines' 'Chad' 'Egypt' 'Indonesia' 'Oman'
 'Kuwait' 'Ukraine' 'Yemen' 'Democratic Republic of the Congo' 'Mali'
 'Saudi Arabia' 'Qatar' 'China' 'Congo (Democratic Republic)' 'Jordan'
 'Norway' 'Bangladesh' 'Belgium' 'Bosnia and Herzegovina' 'Malaysia'
 'Niger' 'Mauritania' 'Morocco' 'Occupied Palestinian Territories' 'Gaza'
 'Albania' 'Ethiopia' 'Netherlands' 'Comoros' 'Tanzania'
 'Trinidad and Tobago' 'Eritrea' 'Sweden' 'Uganda'
 'Palestinian Territories' 'South Africa' 'France' 'Venezuela' 'Nigeria'
 'Nicaragua' 'UK' 'Liechtenstein' 'Serbia' 'Austria' 'Rwanda' 'The Gambia'
 'Ireland' 'Hai

In [207]:
country_corrections = {
    'UK': 'United Kingdom',
    'United States of America': 'United States',
    'DPRK': 'North Korea',
    'Syrian Arab Republic': 'Syria',
    'Russian Federation (as at November 2010)': 'Russia',
    'RUSSIA': 'Russia',
    'Iraq (previous address)': 'Iraq',
    'Occupied Palestinian Territories': 'Palestinian Territories',
    'Gaza': 'Palestinian Territories',
    'Türkiye': 'Turkey',
    'Congo (Democratic Republic)': 'Democratic Republic of the Congo',
    'Moscow': 'Russia',
    '115088': np.nan
}
df['Country'] = df['Country'].replace(country_corrections)
invalid_countries = ['115088', 'Moscow', None, np.nan]
df['Country'] = df['Country'].where(~df['Country'].isin(invalid_countries), np.nan)

df['Country'] = df['Country'].apply(lambda x: x if isinstance(x, str) and len(x.strip()) > 2 else np.nan)



In [208]:
print("Valores únicos después de limpieza:", df['Country'].unique())
print("Total NaNs en 'Country':", df['Country'].isna().sum())

Valores únicos después de limpieza: ['Pakistan' nan 'Kazakhstan' 'Syria' 'Myanmar' 'Belarus' 'Russia'
 'United Kingdom' 'North Korea' 'Singapore' 'United Arab Emirates' 'Iran'
 'Libya' 'United States' 'Lebanon' 'Tunisia' 'Greece'
 'Central African Republic' 'Germany' 'Iraq' 'Italy' 'Algeria' 'Turkey'
 'Sudan' 'Afghanistan' 'Somalia' 'Kenya' 'Philippines' 'Chad' 'Egypt'
 'Indonesia' 'Oman' 'Kuwait' 'Ukraine' 'Yemen'
 'Democratic Republic of the Congo' 'Mali' 'Saudi Arabia' 'Qatar' 'China'
 'Jordan' 'Norway' 'Bangladesh' 'Belgium' 'Bosnia and Herzegovina'
 'Malaysia' 'Niger' 'Mauritania' 'Morocco' 'Palestinian Territories'
 'Albania' 'Ethiopia' 'Netherlands' 'Comoros' 'Tanzania'
 'Trinidad and Tobago' 'Eritrea' 'Sweden' 'Uganda' 'South Africa' 'France'
 'Venezuela' 'Nigeria' 'Nicaragua' 'Liechtenstein' 'Serbia' 'Austria'
 'Rwanda' 'The Gambia' 'Ireland' 'Haiti' 'Laos' 'South Sudan'
 "Democratic People's Republic of Korea" 'Thailand' 'Estonia' 'Cambodia'
 'Uzbekistan' 'Switzerland' 'Marsh

In [211]:
print("Valores únicos después de nacionalidad", df['Nationality'].unique())
print("Total NaNs en 'Nationality':", df['Nationality'].isna().sum())

Valores únicos después de nacionalidad ['Pakistan' 'Russia' nan 'Tanzania' 'Afghanistan' 'Saudi Arabia' 'Libya'
 'Syria' 'Tunisia' 'Iran' 'Central African Republic'
 '(1) Germany. (2) Morocco' 'Iraq' 'Jordan' 'Algeria' 'Sudan'
 '(1) Central African Republic. (2) South Sudan' 'Egypt' 'Eritrea'
 'Somalia' '(1) Somalia (2) Kenya' 'Philippines'
 '(1) Central African Republic. (2) Chad' 'India' 'Indonesia' 'Kuwait'
 'Yemen' 'Norway' 'Ukraine' 'Uganda' '(1) Russia (2) Israel (3) Portugal'
 'Mauritania' 'Palestinian' '(1) Pakistan. (2) Saudi Arabia'
 'United Republic of Tanzania' 'Qatar' '(1) Russia. (2) Ukraine' 'Rwanda'
 'United Kingdom' 'Mali' '(1) Afghanistan (2) Pakistan'
 '(1) Lebanon (2) Belgium' '(1) Syria. (2) Palestine'
 '(1) Syria. (2) United Kingdom' '(1) Germany (2) Algeria'
 '(1) Algeria. (2) Palestine' 'France' '(1) Russia. (2) Syria' 'Morocco'
 '(1) Malaysia (2) Indonesia' '(1) France. (2) Tunisia'
 '(1) United States (2) Syria' '(1) Jordan. (2) United States'
 '(1) United Sta

In [240]:
df['Nationality'][-56:]

18740            [Tunisia]
18741            [Tunisia]
18742            [Tunisia]
18743        [Netherlands]
18744        [Afghanistan]
18745                   []
18746                   []
18747                   []
18748                   []
18749            [Lebanon]
18750               [Iraq]
18751        [Netherlands]
18752                   []
18753             [Russia]
18754            [Somalia]
18755    [Lebanon., Syria]
18756    [Lebanon., Syria]
18757             [Russia]
18758            [Belarus]
18759            [Belarus]
18760            [Belarus]
18761            [Belarus]
18762             [Russia]
18763             [Russia]
18764           [Malaysia]
18765           [Malaysia]
18766              [Syria]
18767              [Syria]
18768              [Syria]
18769                   []
18770                   []
18771                   []
18772            [Belarus]
18773            [Belarus]
18774          [Indonesia]
18775          [Indonesia]
18776             [Russia]
1